# 4th lab:
___
***prepared by Pavel 3P & Markov Arthur***

#### *TODO*:

1. Pansharpening methods analysing.

### Area info:

In [1]:
bottom_left = (30.073321247506765, 49.345775018245774)
upper_right = (32.986007792928522, 50.778667808079206)

#Filtering options:
from_time = "2017-04-01T00:00:00Z"
to_time = "2020-05-01T00:00:00Z"
maxCloudCoverage = 0
mosaickingOrder = "leastRecent"

#Image size in px: (2500px is max)
width = 2500
height = 2500

## First part:

### Create session with server:

In [58]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

def get_token():
    client_id = "8d7432b3-19c9-4a6e-9d02-fcf865d1a6c6"
    client_secret = "fdR{0FFu(}F)KD/s@>9(w;28*]MHkQJoo_I!*Q73"


    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)


    token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                              client_id=client_id, client_secret=client_secret)
    return token['access_token']

token = get_token()

<span style="font-size: 115%; font-family: courier new">
    <span style="color: green"><b>def </b></span>
    <span style="color: blue">get_image</span>
    (satellite: <span style="color: green">str</span>, 
    bands, 
    merge=<span style="color: green"><b>True</b></span>, 
    img_name: <span style="color: green">str</span> ="image", 
    img_format: <span style="color: green">str</span> ="tiff"):
</span>

___
*satellite*: str. Satellite name (sentinel/landsat).

*bands*: list or int. Bands to download.

*merge*=__True__: bool. If it's __True__ merges all bands in one image. If it's __False__ downloads bands separately and saves them in it's individual file.

*img_name*="image": str or list. Name of image. If merge=__False__ it may be ignored or it must be a list of names with length equal to length of the *bands*.

*img_format*="tiff": str. Image format.

In [114]:
import requests
import os
import json
import winsound


def get_image(satellite: str, bands, merge=True, img_name: str ="image", img_format: str ="tiff"):
    if not os.path.isdir("data"):
        os.mkdir("data")  
    if not os.path.isdir(f"data/{satellite}"):
        os.mkdir(f"data/{satellite}")
        
    if type(bands) is not type(list()):
        bands = [bands]
    
    
    band_names = {
        4: "r", 
        3: "g", 
        2: "b", 
        8: "b08",
    }
    band_id = {
        4:"B04",
        3:"B03",
        2:"B02",
        8:"B08",
    }
    satellite_id = {
        "sentinel": "S2L2A",
        "landsat": "L8L1C",
    }
    urls = {
        "sentinel": "https://services.sentinel-hub.com/api/v1/process",
        "landsat": "https://services-uswest2.sentinel-hub.com/api/v1/process",
    }
    
    
    if not merge:
        if type(img_name) is type(list()):
            names = img_name
        else:
            names = band_names
            
        for name, band in zip(names, bands):
            get_image(satellite, band, img_format=img_format, merge=True, img_name=name)
        return

    
    evaluate_pixel_output = "["
    for i in list(band_id.keys()):
        if i in bands:
            evaluate_pixel_output += "2.5 * sample." + band_id[i] + ", "
    evaluate_pixel_output = evaluate_pixel_output[:-2] + "]"
    
    
    evalscript = """
        //VERSION=3

        function setup() {
          return {
            input: """ + str([band_id[i] for i in bands]) + """, 
            output: {
              bands: """ + str(len(bands)) + """
            }
          };
        }

        function evaluatePixel(
          sample,
          scenes,
          inputMetadata,
          customData,
          outputMetadata
        ) {
          return """ + evaluate_pixel_output + """; 
        }
        """
    
    # Checking token:
    try:
        resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")
    except Exception as e:
        #if e.error == "token_expired":
        token = get_token()    
        
    req_data = {
        "input": {
            "bounds": {
                "bbox": [
                    bottom_left[0],
                    bottom_left[1],
                    upper_right[0],
                    upper_right[1],
                ]
            },
            "data": [
                {
                    "type": satellite_id[satellite],
                    "dataFilter": {
                        "timeRange": {
                            "from": from_time,
                            "to": to_time,
                        },
                        "maxCloudCoverage": maxCloudCoverage,
                        "mosaickingOrder": mosaickingOrder,
                    },
                }
            ]
        },
        "output": {
            "width": width,
            "height": height,
            "responses": [{
                "format": {
                    "type": "image/" + img_format
                }
            }]
        },
        "evalscript": evalscript
    }
    headers = {
        "Authorization": f"Bearer {token}",
    }
    url = urls[satellite]
            

    response = requests.post(url, json=req_data, headers=headers)
    
    if response.status_code != 200:
        info = json.loads(response.text)
        for i in info["error"].items():
            print(i)
    else:
        with open(f"data/{satellite}/{img_name}.{img_format}", "wb") as img:
            img.write(response.content)
    
    winsound.MessageBeep(type=winsound.MB_ICONHAND)

## Second part:


## Third part:

### Getting all necessary bands:

In [7]:
get_image("landsat", bands=[2, 3, 4], img_name="color")
get_image("landsat", bands=8, img_name="pchrc")

### Degrading image:

In [20]:
!gdal_translate -outsize 200% 200%  data\landsat\color.tiff data\landsat\color_degraded.tiff
!gdal_translate -outsize 200% 200%  data\landsat\pchrc.tiff data\landsat\pchrc_degraded.tiff

Input file size is 2500, 2500
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 2500, 2500
0...10...20...30...40...50...60...70...80...90...100 - done.


### Pansharpening:

In [47]:
!python gdal\gdal_pansharpen.py -r average data\landsat\pchrc_degraded.tiff data\landsat\color_degraded.tiff data\landsat\pansharpened.tiff
!gdal_translate -outsize 50% 50%  data\landsat\pansharpened.tiff data\landsat\pansharpened_degraded.tiff

0...10...20...30...40...50...60...70...80...90...100 - done.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal

Input file size is 5000, 5000
0...10...20...30...40...50...60...70...80...90...100 - done.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugi

### Calculating accuracy of pansharpening:

In [43]:
import gdal
import numpy as np
from sklearn.metrics import accuracy_score as accuracy

orig = gdal.Open("data\landsat\color.tiff").ReadAsArray().astype(np.float32)
pshd = gdal.Open("data\landsat\pansharpened_degraded.tiff").ReadAsArray().astype(np.float32)

orig = orig.reshape(orig.size)
pshd = pshd.reshape(pshd.size)

accuracy(orig, pshd)

0.017678133333333332

# TIME FOR EXPERIMENTS!!!11!

In [140]:
bottom_left = (39.56888717651368, 47.951004999484974)
upper_right = (39.76888717651368, 48.151004999484974)

#Filtering options:
from_time = "2017-04-01T00:00:00Z"
to_time = "2020-05-01T00:00:00Z"
maxCloudCoverage = 0
mosaickingOrder = "leastRecent"

#Image size in px: (2500px is max)
width = 2500
height = 2500

In [141]:
get_image("landsat", bands=[2, 3, 4], img_name="color")

In [142]:
get_image("landsat", bands=8, img_name="pchrc")

!gdal_translate -outsize 200% 200%  data\landsat\color.tiff data\landsat\color_degraded.tiff
!gdal_translate -outsize 200% 200%  data\landsat\pchrc.tiff data\landsat\pchrc_degraded.tiff

!python gdal\gdal_pansharpen.py -r average data\landsat\pchrc_degraded.tiff data\landsat\color_degraded.tiff data\landsat\pansharpened.tiff

Input file size is 2500, 2500
0...10...20...30...40...50...60...70...80...90...100 - done.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugi

Input file size is 2500, 2500
0...10...20...30...40...50...60...70...80...90...100 - done.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugi

0...10...20...30...40...50...60...70...80...90...100 - done.


ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.

ERROR 1: Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal